<a href="https://colab.research.google.com/github/yejijang-analyst/stockmarket-prediction/blob/main/%EB%84%A4%EC%9D%B4%EB%B2%84%EA%B8%B0%EC%82%AC%ED%81%AC%EB%A1%A4%EB%A7%81_%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2023년 증권탭 크롤링 및 감성분석 일부

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

def crawl_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    titles_html = soup.select('#wrap #newarea #contentarea #contentarea_left .newsSchResult._replaceNewsLink dl .articleSubject a')
    dates_html = soup.select('#wrap #newarea #contentarea #contentarea_left .newsSchResult._replaceNewsLink dl dd span.wdate')

    data = []
    for title, date in zip(titles_html, dates_html):
        date_text = date.text.strip()  # Add strip() to remove whitespaces
        title_text = title.text.strip()
        data.append({'날짜': date_text, '기사 제목': title_text})

    return data

# 시작 페이지와 끝 페이지 설정
start_page = 1
end_page = 119  # 예시로 2페이지까지 크롤링

# 모든 데이터를 담을 빈 데이터프레임 생성
dfs = []

# 페이지 순회하며 크롤링
for page in range(start_page, end_page + 1):
    page_url = f"https://finance.naver.com/news/news_search.naver?rcdate=&q=%C4%DA%BD%BA%C7%C7&x=0&y=0&sm=all.basic&pd=4&stDateStart=2023-01-01&stDateEnd=2023-01-15&page={page}"
    page_data = crawl_page(page_url)
    dfs.append(pd.DataFrame(page_data))

# concatenate the list of DataFrames
df = pd.concat(dfs, ignore_index=True)

# 데이터프레임의 '날짜' 열을 datetime 형식으로 변환
df['날짜'] = df['날짜'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M"))

# 데이터프레임에서 '날짜' 열의 중간 공백 삭제
df['날짜'] = df['날짜'].dt.strftime('%Y-%m-%d %H:%M')
df['날짜'] = df['날짜'].str.replace('\s+', ' ', regex=True)

# 데이터프레임 출력
print(df)
print(len(df))

                    날짜                                             기사 제목
0     2023-01-15 21:49                            돌아온 외인…올 들어 2조8839억 매수
1     2023-01-15 19:32                  기준금리 인상 ‘끝’ 보이지만… 영끌족 대출이자 ‘눈덩이’
2     2023-01-15 19:09  15년째 문턱 못넘은 선진 지수… 정부 더 적극적으로 나서야 [김기석의 자본시장 ...
3     2023-01-15 19:04            4만→6만→8만원?, 카카오 팔았더니 오른다 “폭락때보다 더 속터져”
4     2023-01-15 18:45       美주식·금·채권 '팩터기반 운용'… 하락장 방어력 과시 [이런 펀드 어때요?]
...                ...                                               ...
2361  2023-01-01 06:41                  하락장에 그룹 시총 순위 요동…LG·포스코↑ SK·네이버↓
2362  2023-01-01 06:01            [2023 증시전망] ②올해 코스피, 반도체·이차전지·바이오 ‘맑음’
2363  2023-01-01 06:01           [2023 증시전망] ①증권가 “상반기 코스피 2135~2550 전망”
2364  2023-01-01 06:00                    [주간증시전망]'1월 효과' 나타날까…美 고용지표 주목
2365  2023-01-01 00:01             "반갑다, 2023년!"…국내증시, '산타' 갔지만 '까치' 올까?

[2366 rows x 2 columns]


In [ ]:
def remove_words(title, target_words):
    for word in target_words:
        title = title.replace(word, '')
    return title

# target_words 설정
target_words = ['주간증시전망', '코스피', '주가전망',"코스닥", "마감", "시황","증시","주간전망","증시전망"]

# 'title' 열에 대해 remove_words 함수 적용
df['기사 제목2'] = df['기사 제목'].apply(lambda x: remove_words(x, target_words))

In [ ]:
df.head()

,날짜,기사 제목,기사 제목2
0,2023-01-15 21:49,돌아온 외인…올 들어 2조8839억 매수,돌아온 외인…올 들어 2조8839억 매수
1,2023-01-15 19:32,기준금리 인상 ‘끝’ 보이지만… 영끌족 대출이자 ‘눈덩이’,기준금리 인상 ‘끝’ 보이지만… 영끌족 대출이자 ‘눈덩이’
2,2023-01-15 19:09,15년째 문턱 못넘은 선진 지수… 정부 더 적극적으로 나서야 [김기석의 자본시장 ...,15년째 문턱 못넘은 선진 지수… 정부 더 적극적으로 나서야 [김기석의 자본시장 ...
3,2023-01-15 19:04,"4만→6만→8만원?, 카카오 팔았더니 오른다 “폭락때보다 더 속터져”","4만→6만→8만원?, 카카오 팔았더니 오른다 “폭락때보다 더 속터져”"
4,2023-01-15 18:45,美주식·금·채권 '팩터기반 운용'… 하락장 방어력 과시 [이런 펀드 어때요?],美주식·금·채권 '팩터기반 운용'… 하락장 방어력 과시 [이런 펀드 어때요?]


In [ ]:
x = df["기사 제목2"].tolist()
y = [result["label"] for result in kor_clf_sentiment(x)]
y2 = [result["score"] for result in kor_clf_sentiment(x)]

df1_1 = pd.DataFrame(data={"date": df["날짜"],"title": df["기사 제목"],"text": x, "sentiment": y, "score": y2})

df1_1.to_csv("11월 코스피.csv",encoding='utf-8-sig',index=True)

거래일자 전처리 코드

In [ ]:
# 필요함수 정의

import pandas as pd
from datetime import datetime, timedelta

# 1. 주말 및 평일에 따라 날짜 조정하는 함수
def adjust_date(row):
    if row['date'].weekday() < 5:  # 평일인 경우
        if row['date'].hour < 15:
            return row['date'].strftime('%Y-%m-%d')
        else:
            if row['date'].weekday() == 4: # 금요일 15시 지난 경우는 월요일로 수정
              return (row['date'] + timedelta(days=3)).strftime('%Y-%m-%d')
            else:
              return (row['date'] + timedelta(days=1)).strftime('%Y-%m-%d')

    else:  # 주말인 경우
        if row['date'].weekday() == 5:  # 토요일
            return (row['date'] + timedelta(days=2)).strftime('%Y-%m-%d')
        else:  # 일요일
            return (row['date'] + timedelta(days=1)).strftime('%Y-%m-%d')

# 2. 2023년 공휴일 휴장일 다음 개장일자로 여는 코드

import pandas as pd

def modify_holiday(df):
    # 'Date' 열을 datetime 형식으로 변환
    df['date_adjusted'] = pd.to_datetime(df['date_adjusted'], format='%Y-%m-%d')

    # 날짜 수정
    df['date_adjusted'] = df['date_adjusted'].apply(lambda x: x.replace(day=x.day + 1) if x.day in [24] and x.month == 1 else x)
    df['date_adjusted'] = df['date_adjusted'].apply(lambda x: x.replace(day=x.day + 2) if x.day == 23 and x.month == 1 else x)
    df['date_adjusted'] = df['date_adjusted'].apply(lambda x: x.replace(day=x.day + 1) if x.day == 1 and x.month == 3 else x)
    df['date_adjusted'] = df['date_adjusted'].apply(lambda x: x.replace(day=x.day + 1) if x.day == 1 and x.month == 5 else x)
    df['date_adjusted'] = df['date_adjusted'].apply(lambda x: x.replace(day=x.day + 3) if x.day == 5 and x.month == 5 else x)
    df['date_adjusted'] = df['date_adjusted'].apply(lambda x: x.replace(day=x.day + 1) if x.day == 6 and x.month == 6 else x)
    df['date_adjusted'] = df['date_adjusted'].apply(lambda x: x.replace(day=x.day + 1) if x.day == 15 and x.month == 8 else x)
    df['date_adjusted'] = df['date_adjusted'].apply(lambda x: x.replace(day=2, month=10, year=2023) if x.day in [28, 29] and x.month == 9 else x)
    df['date_adjusted'] = df['date_adjusted'].apply(lambda x: x.replace(day=x.day + 1) if x.day == 3 and x.month == 10 else x)
    df['date_adjusted'] = df['date_adjusted'].apply(lambda x: x.replace(day=x.day + 1) if x.day == 9 and x.month == 10 else x)
    df['date_adjusted'] = df['date_adjusted'].apply(lambda x: x.replace(day=x.day + 1) if x.day == 25 and x.month == 12 else x)
    df['date_adjusted'] = df['date_adjusted'].apply(lambda x: x.replace(day=2, month=1, year=2024) if x.day == 29 and x.month == 12 else x)

    return df

In [ ]:
# 데이터 불러오기
news1_1 = pd.read_csv('/content/0101_0115.csv')
news1_2 = pd.read_csv('/content/0116_0131.csv')
news1_3= pd.read_csv('/content/0201_0214.csv')
news1_4 = pd.read_csv('/content/0215_0228.csv')
news1_5 = pd.read_csv('/content/0301_0315.csv')
news1_6 = pd.read_csv('/content/0316_0331.csv')
news1_7 = pd.read_csv('/content/0401_0415.csv')
news1_8 = pd.read_csv('/content/0416_0430.csv')
news1_9 = pd.read_csv('/content/0501_0515.csv')
news1_10 = pd.read_csv('/content/0516_0531.csv')


# 1-5월 데이터 합치기
news1 = pd.concat([news1_1, news1_2,news1_3 ,news1_4 , news1_5,news1_6 ,news1_7 ,news1_8 ,news1_9 ,news1_10  ], ignore_index=True)

In [ ]:
# 6-12월 데이터 불러오기
news2 = pd.read_csv('/content/2023하반기.csv')

# 1-5월 데이터 합치기
news = pd.concat([news1,news2 ], ignore_index=True)

In [ ]:
# 필요없는 칼럼 정리
news = news.drop(['Unnamed: 0'], axis=1)

In [ ]:
news

,date,title,text,sentiment
0,2023-01-15 21:49,돌아온 외인…올 들어 2조8839억 매수,돌아온 외인…올 들어 2조8839억 매수,positive
1,2023-01-15 19:32,기준금리 인상 ‘끝’ 보이지만… 영끌족 대출이자 ‘눈덩이’,기준금리 인상 ‘끝’ 보이지만… 영끌족 대출이자 ‘눈덩이’,neutral
2,2023-01-15 19:09,15년째 문턱 못넘은 선진 지수… 정부 더 적극적으로 나서야 [김기석의 자본시장 ...,15년째 문턱 못넘은 선진 지수… 정부 더 적극적으로 나서야 [김기석의 자본시장 ...,neutral
3,2023-01-15 19:04,"4만→6만→8만원?, 카카오 팔았더니 오른다 “폭락때보다 더 속터져”","4만→6만→8만원?, 카카오 팔았더니 오른다 “폭락때보다 더 속터져”",neutral
4,2023-01-15 18:45,美주식·금·채권 '팩터기반 운용'… 하락장 방어력 과시 [이런 펀드 어때요?],美주식·금·채권 '팩터기반 운용'… 하락장 방어력 과시 [이런 펀드 어때요?],neutral
...,...,...,...,...
52486,2023-06-16 3:04,"개미, 코스닥 거래비중 사상 첫 80% 밑돌아","개미, 거래비중 사상 첫 80% 밑돌아",negative
52487,2023-06-16 3:02,"골프·진단키트·중고차株, 코로나 끝나자 ‘와르르’","골프·진단키트·중고차株, 코로나 끝나자 ‘와르르’",neutral
52488,2023-06-16 3:01,"檢, 제2 SG사태 본격수사… 주식카페 운영자 압수수색","檢, 제2 SG사태 본격수사… 주식카페 운영자 압수수색",negative
52489,2023-06-16 0:03,"애매한 파월, 7월 실적시즌 쏠린 눈…""이익 눈높이 주목""","애매한 파월, 7월 실적시즌 쏠린 눈…""이익 눈높이 주목""",neutral


In [ ]:
# datetype 로 변환
news['date'] = pd.to_datetime(news['date'], format='%Y-%m-%d %H:%M')


# 1. 날짜 조정 적용
news['date_adjusted'] = news.apply(adjust_date, axis=1)



In [ ]:
# 2. 공휴일 변환 적용
news = modify_holiday(news)

In [ ]:
news.sort_values(by='date_adjusted', inplace=True)

In [ ]:
news.to_csv("거래일자처리_최종.csv",encoding='utf-8-sig',index=True)

In [ ]:
# 2024 0102부터 개장으로 0101을 0102 로 대체

news['date_adjusted'] = news['date_adjusted'].apply(lambda x: x.replace(day=x.day + 1) if x.day == 1 and x.month == 1 else x)

In [ ]:
news.to_csv("거래일자처리_정말최종.csv",encoding='utf-8-sig',index=True)

In [ ]:
news

,date,title,text,sentiment,date_adjusted
2179,2023-01-02 11:36:00,"2023년 코스피, 상승 출발","2023년 , 상승 출발",neutral,2023-01-02
2196,2023-01-02 10:55:00,"계묘년 첫날 금융시장 '안정'…장초반 주가 소폭 상승·환율 1,260원대","계묘년 첫날 금융시장 '안정'…장초반 주가 소폭 상승·환율 1,260원대",positive,2023-01-02
2195,2023-01-02 10:56:00,"2023년 주식시장 문 열렸다… 한국거래소, 개장식 개최","2023년 주식시장 문 열렸다… 한국거래소, 개장식 개최",neutral,2023-01-02
2194,2023-01-02 11:00:00,"[시황] 새해 첫 개장일, 코스피 2240선-코스닥 680선 회복","[] 새해 첫 개장일, 2240선- 680선 회복",neutral,2023-01-02
2193,2023-01-02 11:00:00,2023년 코스피 상승 출발,2023년 상승 출발,neutral,2023-01-02
...,...,...,...,...,...
22175,2024-01-12 15:52:00,"기관·외인 매도 언제까지…코스피, 2520선까지 밀려","기관·외인 매도 언제까지…, 2520선까지 밀려",negative,2024-01-15
22174,2024-01-12 15:52:00,"코스피, 8일째 하락 마감…2520선 까지 '털썩'",", 8일째 하락 …2520선 까지 '털썩'",negative,2024-01-15
22173,2024-01-12 15:54:00,"[속보] 코스피, 8거래일 연속 하락…1년8개월來 최장","[속보] , 8거래일 연속 하락…1년8개월來 최장",negative,2024-01-15
22187,2024-01-12 15:44:00,코스피 하락 마감,하락,negative,2024-01-15
